# AUTOMATED BUILD OF MODEL

In [ ]:
import tensorflow as tf

# Settings up u-net model architecture
## down block

In [ ]:
def down_block(down_input, m_2D_convs, n_3D_convs, c_output_channels, pool=True, crop=None):
    
    # RELU ACTIVATION LAYERS
    # x: Input tensor.
    # alpha: float. Slope of the negative part. Defaults to zero.
    # max_value: float. Saturation threshold.
    # threshold: float. Threshold value for thresholded activation.
    
    #down = tf.keras.layers.relu(down_input, alpha=0.0, max_value=None, threshold=0.0)
    down = tf.keras.layers.Activation("relu")(down_input)
    
    # CONVOLUTIONAL LAYERS
    # filters: Integer, the dimensionality of the output space (i.e. the number of output filters in the convolution).
    
    # kernel_size:
    # 2D: An integer or tuple/list of 2 integers, specifying the height and width of the 2D convolution window.
    # Can be a single integer to specify the same value for all spatial dimensions
    # 3D: An integer or tuple/list of 3 integers, specifying the depth, height and width of the 3D convolution window. 
    # Can be a single integer to specify the same value for all spatial dimensions.
    
    # strides:
    # 2D: An integer or tuple/list of 2 integers, specifying the strides of the convolution 
    # along the height and width. Can be a single integer to specify the same value for all spatial dimensions.
    # 3D:  An integer or tuple/list of 3 integers, specifying the strides of the convolution along each spatial 
    # dimension. Can be a single integer to specify the same value for all spatial dimensions.
    
    # padding: One of "valid" or "same" (case-insensitive). "valid" means "no padding". 
    # "same" results in padding the input such that the output has the same length as the original input.
    
    # activation: Activation function to use (see activations). If you don't specify anything, 
    # no activation is applied (ie. "linear" activation: a(x) = x).
    
    # kernel_initializer: Initializer for the kernel weights matrix

    for i in range(m_2D_convs):
        # repeat m times
        
        # 2D convolutional layer
        # 1*3*3 same convolution
        # Rel0 activation
        down = tf.keras.layers.Conv3D(filters=c_output_channels,
                                            kernel_size=(1,3,3),
                                            strides=1,
                                            activation='relu',
                                            kernel_initializer='he_normal',
                                            padding='same')(down)
    
    for i in range(n_3D_convs):
        # repeat loop n-1 times
        
        # 3D convolutional layer
        # 1*3*3 same convolution
        # Rel0 activation
        down = tf.keras.layers.Conv3D(filters=c_output_channels,
                                            # Note: Stated (1,3,3) convs
                                            kernel_size=(1, 3, 3),
                                            strides=1,
                                            activation='relu',
                                            kernel_initializer='he_normal',
                                            padding='same')(down)
        
        if i == n_3D_convs - 1: 
            # 3D convolutional layer
            # 1*3*3 valid convolution
            # No activation
            down = tf.keras.layers.Conv3D(filters=c_output_channels,
                                            # Note: Stated (3,1,1) convs
                                            kernel_size=(3, 1, 1),
                                            strides=1,
                                            activation=None,
                                            kernel_initializer='he_normal',
                                            padding='valid')(down)
        
        else:         
            # Final 3D convolutional layer
            # 1*3*3 valid convolution
            # Rel0 activation
            down = tf.keras.layers.Conv3D(filters=c_output_channels,
                                            # Note: Stated (3,1,1) convs
                                            kernel_size=(3, 1, 1),
                                            strides=1,
                                            activation='relu',
                                            kernel_initializer='he_normal',
                                            padding='valid')(down)


    
    # Crop the image centrally in depth dimension
    # could try tf.keras.layers.Cropping3D if this fails
    if crop == None:
        crop_central_depth = down_input[:]
    else:
        crop_central_depth = down_input[:, crop[0]:crop[1]]
    
    # Adapt channels - 1*1*1 conv to c channels
    adapt_channels = tf.keras.layers.Conv3D(c_output_channels, (1, 1, 1), activation=None)(crop_central_depth)
    
    down = tf.keras.layers.Add()([down, adapt_channels])
    
            
    # POOLING LAYERS 3D
    # pool_size: tuple of 3 integers, factors by which to downscale (dim1, dim2, dim3). (2, 2, 2) will halve the size of the 3D input in each dimension.
    # strides: tuple of 3 integers, or None. Strides values.
    # padding: One of "valid" or "same" (case-insensitive).
    # data_format: A string, one of channels_last (default) or channels_first. The ordering of the dimensions in the inputs. 
    # channels_last corresponds to inputs with shape (batch, spatial_dim1, spatial_dim2, spatial_dim3, channels) while 
    # channels_first corresponds to inputs with shape (batch, channels, spatial_dim1, spatial_dim2, spatial_dim3). 
    # defaults to channels_last
    
    # 3D Average (not max) pooling layer
    # 1*2*2
    if pool == True:
        pool = tf.keras.layers.AveragePooling3D(pool_size=(1, 2, 2), strides=None, padding='valid', data_format='channels_last')(down)
        return down, pool
    else:
        return down

## fc residual block
### TODO: See NOTE below

In [ ]:
def fc_block(down_input, c_output_channels):
    fc = tf.keras.layers.Conv3D(filters=c_output_channels,
                                            kernel_size=(1,8,8),
                                            strides=1,
                                            activation=None,
                                            kernel_initializer='he_normal',
                                            padding='valid')(down_input)
    
    for i in range(2):
        crop = fc[:]
        fc = tf.keras.layers.Activation("relu")(fc)
        
        # NOTE: I believe there should be a fully connected layer here
        fc = tf.keras.layers.Dense(1024)(fc)
        fc = tf.keras.layers.Add()([fc, crop])
    
    fc = tf.keras.layers.Activation("relu")(fc) 
    fc = tf.keras.layers.Reshape((1,8,8,256))(fc)
    
    return fc

## Up block
### TODO: Tensor shapes returned dont upscale correctly - see model assembly output

In [ ]:
def up_block(down_input, up_input, m_2D_convs, c_output_channels, upscale=True):
    
    print("up_block")
    
    print(">>>> upblock arg down_in:", down_input)
    print(">>>> upblock arg up_input:", up_input)
    
    if upscale == True:
        up = tf.keras.layers.UpSampling3D(size=(1,2,2))(up_input)
        print(">>>> upscale layer:", up)
    else:
        up = up_input
    
    up = tf.keras.layers.concatenate([down_input, up], axis=0)
    print(">>>> concat layer:", up)
    
    crop = up[:]
    adapt_channels = tf.keras.layers.Conv3D(c_output_channels, (1, 1, 1), activation=None)(crop)
    
    up = tf.keras.layers.Activation("relu")(up)
    
    for i in range(m_2D_convs):
        up = tf.keras.layers.Conv3D(filters=c_output_channels,
                                            # Note: Stated (1,3,3) convs
                                            kernel_size=(1, 3, 3),
                                            strides=1,
                                            activation='relu',
                                            kernel_initializer='he_normal',
                                            padding='same')(up)
        print(">>>> conv layer:", up)
        
    up = tf.keras.layers.Add()([up, adapt_channels])
    print(">>>> add layer:", up)
    return up

## Model assembly

In [ ]:
def unet_model(input_shape):
    
    # INPUT PATH
    # Input level 1
    input_1 = tf.keras.layers.Input(shape=input_shape)
    
    # ENCODER PATH
    # NOTE: Did not specify strides or padding for pooling
    # Encoder level 1
    down_1, pool_1 = down_block(input_1, 3, 0, 32)
    print("down1:", down_1)
    print("pool1:", pool_1)
    print("---------------------")
    # Encoder level 2
    down_2, pool_2 = down_block(pool_1, 3, 0, 32)
    print("down2:", down_2)
    print("pool2:", pool_2)
    print("---------------------")
    # Encoder level 3
    down_3, pool_3 = down_block(pool_2, 3, 0, 64)
    print("down3:", down_3)
    print("pool3:", pool_3)
    print("---------------------")
    # Encoder level 4
    down_4, pool_4 = down_block(pool_3, 1, 2, 64, crop=(2,19))
    print("down4:", down_4)
    print("pool4:", pool_4)
    print("---------------------")
    # Encoder level 5
    down_5, pool_5 = down_block(pool_4, 1, 2, 128, crop=(4,17))
    print("down5:", down_5)
    print("pool5:", pool_5)
    print("---------------------")
    # Encoder level 6
    down_6, pool_6 = down_block(pool_5, 1, 2, 128, crop=(4,17))
    print("down6:", down_6)
    print("pool6:", pool_6)
    print("---------------------")
    # Encoder level 7 - No pooling
    down_7 = down_block(pool_6, 0, 4, 256, pool=False, crop=(8,13))
    print("down7:", down_7)
    print("---------------------")

    # FULLY CONNECTED PATH - level 8
    fc_8 = fc_block(down_7, 1024)
    print("fc8:", fc_8)
    print("---------------------")

    # DECODER PATH
    # Decoder level 7 - No upscaling
    up_7 = up_block(down_7, fc_8, 4, 256, upscale=False)
    print("up7:", up_7)
    print("---------------------")
    # Decoder level 6
    up_6 = up_block(down_6, up_7, 4, 128)
    print("up6:", up_6)
    print("---------------------")
    # Decoder level 5
    up_5 = up_block(down_5, up_6, 4, 128)
    print("up5:", up_5)
    print("---------------------")
    # Decoder level 4
    up_4 = up_block(down_4, up_5, 3, 64)
    print("up4:", up_4)
    print("---------------------")
    # Decoder level 3
    up_3 = up_block(down_3, up_4, 3, 64)
    print("up3:", up_3)
    print("---------------------")
    # Decoder level 2
    up_2 = up_block(down_2, up_3, 3, 64)
    print("up2:", up_2)
    print("---------------------")
    # Decoder level 1
    up_1 = up_block(down_1, up_2, 3, 64)
    print("up1:", up_1)
    print("---------------------")

    # OUTPUT PATH
    # Output level 1
    # 1x1x1 sigmoid activation
    output_1 = tf.keras.layers.Conv3D(1, (1, 1, 1), activation='sigmoid')(up_1)
    print("output", output_1)
    print("---------------------")
    
    return tf.keras.Model(inputs=input_1, outputs=output_1)


In [ ]:
input_shape = (21, 512, 512,1)
model = unet_model(input_shape)

## Compile model

In [ ]:
OPTIMIZER = 'adam'
LOSS = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
METRICS = ['accuracy']

model.compile(optimizer = OPTIMIZER, loss = LOSS, metrics = METRICS)

## Plot model

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

## Training model
### TODO: dicom_process.load_data() - see dicom_process_example TODOs

In [ ]:
(train_images, train_labels), (test_images, test_labels) = dicom_process.load_data()
print_data_split(train_images, test_images)

In [ ]:
EPOCHS = 10
BATCH_SIZE = 64

# Fit data to model
model.fit(input_train, target_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=verbosity,
          validation_split=validation_split)

In [ ]:
# Note if model performs better on training than test this is likely overfitting
# Overfitting = memorising training data - does not generalise to other unseen cases
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)